# Tes Code

## Library

In [184]:
import typer
import pandas as pd
import re
from dateutil import parser
import numpy as np
from functools import lru_cache
import pycountry
from spellchecker import SpellChecker
import pkg_resources
from symspellpy import SymSpell, Verbosity


## Load Dataset

In [185]:
df1 = pd.read_csv("Raw Data/custdirty_data.csv")
df2 = pd.read_csv("Raw Data/government_citizenship_dirty.csv")
df3 = pd.read_csv("Raw Data/hospital_data_dirty.csv")

## Read Dataset

In [186]:
df1

,ID,CustomerID,Product_Category,Purchase_Date,Price,Quantity,Product_Description,Country,Phone_Number,Email
0,1,CUST_001,Electronics,Mar 05 2024,65.58,11.0,"Today, federal, claim, report, consider, publi...",USA,250518 1271,eric82@example.com
1,2,CUST_002,Electronics,May 29 2024,335.85,1.0,Alone appear example.,Canada,891.222.8992x0833,allentodd@example.org
2,3,CUST_003,Electronics,May 28 2024,106.92,19.0,"DIRECTION, SECURITY, WHO, GROUND, TRIP, ROOM. ...",United Kingdom,(926)729-9917x4080,oschultz@example.com
3,4,CUST_004,Clothing,Jun 30 2024,389.70,6.0,"Answer, huge, six, new, tough. - picture",Canada,(421)698-4215x479,ajohnson@example.org
4,5,CUST_005,Furniture,14/10/2024,140.78,14.0,Recent last economic establish understand.,Canada,294283 6286,andersonsheriexample.org.
...,...,...,...,...,...,...,...,...,...,...
10295,7431,CUST_7431,Furniture,Feb 24 2024,445.08,6.0,"Meeting, adult, choose, customer, see, base. -...",USA,785235 4238x89420,erinleon@example.net
10296,940,CUST_940,Clothing,Jun 13 2024,188.90,9.0,Actually home blue get.,Usa,380 450 1416x0693,obrienmichael@example.org
10297,9519,CUST_9519,Clothing,Apr 04 2024,371.88,NaN,Option stand determine course include.,Usa,434 895 9812,brownjames@example.net
10298,1804,CUST_1804,Books,May 06 2024,165.60,18.0,"ACCOUNT, AFFECT, NIGHT. - ATTACK",CAN,8682519899,thomas86@example.com


In [187]:
df2

,CitizenID,Name,BirthDate,Nationality,Address,VotingStatus,PassportNumber,TaxID,MaritalStatus,CriminalRecord,EducationLevel,Income,ImmigrationYear
0,CIT_001,Allen Novak,1968-06-16,USA,"597 Carr Creek Apt. 464; West Andrew; NE 76236,",No,460200468,NaN,Married,No,HS,42533.54,NaN
1,CIT_002,TAMARA RAMSEY,1987-02-11,FRANCE,"3899 Medina Roads; Crystaltown; NY 04112,",No,INVALID_our,TAX-002-36,S,No,Masters,28175.78,NaN
2,CIT_003,David Williams,1937-12-15,Bharat,"03064 Eric Ferry; Port Rebecca; AZ 09961,",Y,E48182721,TAX-003-03,Widowed,,High School,143299.35,1986.0
3,CIT_004,Desiree Willis,1991-03-24,USA,"773 Perry Flats; East Michael; WY 77221,",Y,O98365870,TAX_004_11,Married,No,Doctorate,105629.33,NaN
4,CIT_005,Terry Gill,2000-10-15,FR,"214 Michael Crescent Suite 393, New Paulport, ...",N,R72171677,TAX-005-00,S,Pending,PhD,117985.07,2005.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10495,CIT_3673,Scott Barrett,1950-03-08,IND,"10041 Fisher Row Suite 135, Lake Anneburgh, GA...",N,407208444,NaN,Unknown,,PhD,99415.01,NaN
10496,CIT_8693,JAMES BROWN,2000-12-13,US,"563 Romero Ferry, Petersonport, OR 06511",N,983829316,TAX-8693-20,Unknown,NaN,PhD,128327.81,NaN
10497,CIT_7140,Susan Beck,2001-05-11,US,"31275 Michelle Well Suite 725, West Randytown,...",,INVALID_enjoy,TAX-7140-50,Single,Yes,High School,26909.17,NaN
10498,CIT_779,JACKSON STOUT,1983-08-18,India,"10300 Michelle Passage, Amandaburgh, RI 93986",Registered,O91186652,TAX-779-15,S,No,High School,132278.13,1969.0


In [188]:
df3

,PatientID,AdmissionDate,DischargeDate,Diagnosis,Treatment,Doctor,Department,Age,Gender,BloodType,Medication,TestResults,BillingAmount,InsuranceProvider,AppointmentDate
0,PT_001,2024-10-03,2025-05-11,Appendicitis,"Special with job,",Carl Beard,Orthopedics,81,Female,B+,NaN,85/67,9495.73,NaN,2024-02-17
1,PT_K_2,2023-06-27,2023-12-05,Fracture,"Certainly way ten,",Mary Sanchez,Endo,-88,M,O-,civil,140/118,8681.08,HC Insurance,2023-08-08
2,PT_003,2024-05-18,2025-04-29,Appendicitis,"Study small,",Mr. Patrick Harris MD,Orthopedics,94,M,O-,word,92/95,2829.42,NaN,2024-05-21
3,PT_004,Mar 11 1978,2019-10-26,COPD,"Investment human,",Donald Roach,Endocrinology,26,F,A+,NaN,140/119,1121.27,NaN,2024-11-03
4,PT_005,2024-11-23,2024-12-02,Diabetes,"Every coach,",NaN,Ortho,35,Female,B-,NaN,71/68,5555.75,HC Insurance,2023-11-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10295,PT_691,2024-03-16,NaN,COPD,"Focus population green,",Jennifer Rodriguez,Cardio,68,Female,O+,step,High,5014.95,HealthPlus,2024-04-21
10296,PT_7247,2023-03-29,2024-10-27,Fracture,"Hold,",Christine Morris,Ortho,15,F,O-,floor 1 units,98/68,8152.91,NaN,2023-12-14
10297,PT_4814,Oct 11 1998,2005-04-22,Fracture,"Imagine power PM,",Luis May,Endocrinology,3,Male,A+,senior,181/80,2323.13,NaN,2024-06-13
10298,PT_8633,2024-02-01,2024-11-28,Dyabetes,"Morning body than,",Jamie King,Cardiology,52,M,AB+,NaN,169/46,3466.39,HealthPlus,2024-06-26


In [189]:
df3['Diagnosis'].unique()

array(['Appendicitis', 'Fracture', 'COPD', 'Diabetes', 'Hypertension',
       'copd', 'appendicitis', 'Hypertensyon', 'Dyabetes', 'Appendycytys',
       'hypertension', 'fracture', 'diabetes'], dtype=object)

## Missing Value

In [190]:
print(df1.isna().sum())

ID                        0
CustomerID              514
Product_Category       1309
Purchase_Date             0
Price                     0
Quantity                765
Product_Description       0
Country                   0
Phone_Number              0
Email                     0
dtype: int64


In [191]:
print(df2.isna().sum())

CitizenID             0
Name                  0
BirthDate             0
Nationality           0
Address               0
VotingStatus          0
PassportNumber        0
TaxID              1082
MaritalStatus         0
CriminalRecord     2093
EducationLevel        0
Income                0
ImmigrationYear    7893
dtype: int64


In [192]:
print(df3.isna().sum())

PatientID               0
AdmissionDate           0
DischargeDate        3052
Diagnosis               0
Treatment               0
Doctor               1059
Department              0
Age                     0
Gender                  0
BloodType               0
Medication           4151
TestResults             0
BillingAmount           0
InsuranceProvider    3685
AppointmentDate         0
dtype: int64


## Preprocessing

### Kolom ID

In [193]:
len(df1)

10300

In [194]:
def apakah_kolom_id(kolom):
    pattern = r'(id)[_A-Z]?'
    return re.search(pattern, kolom, re.IGNORECASE)

def identifikasi_kolom_id(df, threshold = 0.7):
    kolom_id = []
    for kolom in df.columns:
        ratio = df[kolom].nunique() / len(df)
        if (ratio >= threshold) and apakah_kolom_id(kolom):
            kolom_id.append(kolom)
    return kolom_id

print(f"DF 1 {identifikasi_kolom_id(df1)}")
print(f"DF 2 {identifikasi_kolom_id(df2)}")
print(f"DF 3 {identifikasi_kolom_id(df3)}")

DF 1 ['ID', 'CustomerID']
DF 2 ['CitizenID', 'TaxID']
DF 3 ['PatientID']


### Drop Kolom

In [195]:
kolom_id1 = identifikasi_kolom_id(df1)
kolom_id2 = identifikasi_kolom_id(df2)
kolom_id3 = identifikasi_kolom_id(df3)

# df11 = df1.copy()
# df22 = df2.copy()
# df33 = df3.copy()

df1 = df1.drop(columns=kolom_id1)
df2 = df2.drop(columns=kolom_id2)
df3 = df3.drop(columns=kolom_id3)

print(df1.isna().sum())
print(df2.isna().sum())
print(df3.isna().sum())

Product_Category       1309
Purchase_Date             0
Price                     0
Quantity                765
Product_Description       0
Country                   0
Phone_Number              0
Email                     0
dtype: int64
Name                  0
BirthDate             0
Nationality           0
Address               0
VotingStatus          0
PassportNumber        0
MaritalStatus         0
CriminalRecord     2093
EducationLevel        0
Income                0
ImmigrationYear    7893
dtype: int64
AdmissionDate           0
DischargeDate        3052
Diagnosis               0
Treatment               0
Doctor               1059
Department              0
Age                     0
Gender                  0
BloodType               0
Medication           4151
TestResults             0
BillingAmount           0
InsuranceProvider    3685
AppointmentDate         0
dtype: int64


### Format Nama

In [196]:
def apakah_kolom_nama(kolom):
    pattern = r'(name|nama|pengguna)[_A-Z]?'
    return re.search(pattern, kolom, re.IGNORECASE)

def identifikasi_kolom_nama(df, threshold = 0.7):
    kolom_id = []
    for kolom in df.columns:
        ratio = df[kolom].nunique() / len(df)
        if (ratio >= threshold) and apakah_kolom_nama(kolom):
            kolom_id.append(kolom)
    return kolom_id
    
kolom_nama = identifikasi_kolom_nama(df2)

def format_name(name):
    return name.title()

for kolom in kolom_nama:
    df2[kolom] = df2[kolom].apply(format_name)

df2['Name']

0           Allen Novak
1         Tamara Ramsey
2        David Williams
3        Desiree Willis
4            Terry Gill
              ...      
10495     Scott Barrett
10496       James Brown
10497        Susan Beck
10498     Jackson Stout
10499         Dawn Lane
Name: Name, Length: 10500, dtype: object

### Format Tanggal

In [197]:
df1['Purchase_Date']

0        Mar 05 2024
1        May 29 2024
2        May 28 2024
3        Jun 30 2024
4         14/10/2024
            ...     
10295    Feb 24 2024
10296    Jun 13 2024
10297    Apr 04 2024
10298    May 06 2024
10299     2024-08-12
Name: Purchase_Date, Length: 10300, dtype: object

In [198]:
df2["BirthDate"]

0        1968-06-16
1        1987-02-11
2        1937-12-15
3        1991-03-24
4        2000-10-15
            ...    
10495    1950-03-08
10496    2000-12-13
10497    2001-05-11
10498    1983-08-18
10499    2002-12-23
Name: BirthDate, Length: 10500, dtype: object

In [199]:
if df2["BirthDate"].dtype == "datetime64[ns]":
    print("mantap")
# df1['updateorder'] = np.random.randint(1, 101, size=len(df1))
# df1['purchasedate'] = np.random.randint(1, 101, size=len(df1))
# df1['updateorder']

In [200]:
def apakah_kolom_date(kolom):
    """
    Apakah Kolom Tanggal?
    Pola kata dari kolom yang ingin dihapus
    Mencari kolom dengan pola kata "date" dll
    """
    pattern1 = r'(date|tanggal|tgl|dt)\b|_(date|tanggal|tgl|dt)[_A-Z]?'
    verif1 = re.search(pattern1, kolom, re.IGNORECASE)
    # verif2 = re.search(pattern2, kolom, re.IGNORECASE)
    return verif1
    # return re.search(pattern, kolom, re.IGNORECASE)

def identifikasi_kolom_date(df):
    """
    Identifikasi Kolom ID
    """
    kolom_id = []
    for kolom in df.columns:
        if apakah_kolom_date(kolom):
            kolom_id.append(kolom)
    return kolom_id

for x in df1.columns:
    ratio = df1[x].nunique() / len(df1)
    print(ratio)
    print(apakah_kolom_date(x))
print(identifikasi_kolom_date(df1))
print(identifikasi_kolom_date(df2))
print(identifikasi_kolom_date(df3))

0.0006796116504854369
None
0.10660194174757281
<re.Match object; span=(8, 13), match='_Date'>
0.8632038834951457
None
0.0018446601941747574
None
0.970873786407767
None
0.0006796116504854369
None
0.970873786407767
None
0.9544660194174758
None
['Purchase_Date']
['BirthDate']
['AdmissionDate', 'DischargeDate', 'AppointmentDate']


In [201]:
formats = [
    '%d/%m/%y',  # dd/mm/yy
    '%d/%m/%Y',  # dd/mm/yyyy
    '%Y-%m-%d',  # yyyy-mm-dd
    '%b %d %Y',  # Mmm dd yyyy
    '%d %B %Y'   # dd Mmmm yyyy
]

def try_parsing_date(date_str, formats):
    for fmt in formats:
        try:
            return pd.to_datetime(date_str, format=fmt)
        except ValueError:
            continue
    return pd.NaT
    
def safe_parse(date_str):
    date_str = str(date_str).strip()
    try:
        return parser.parse(date_str)
    except (ValueError, TypeError): 
        return pd.NaT 
# kolom_date = identifikasi_kolom_date(df1)
# for kolom in kolom_date:
#     df1[kolom] = df1[kolom].apply(parser.parse)
kolom_date1 = identifikasi_kolom_date(df1)
kolom_date2 = identifikasi_kolom_date(df2)
kolom_date3 = identifikasi_kolom_date(df3)

print(kolom_date1)
print(kolom_date2)
print(kolom_date3)

# df2['BirthDate'] = df2['BirthDate'].apply(safe_parse)
df2['BirthDate'] = df2['BirthDate'].apply(safe_parse)
# df2['BirthDate']

# df1["Purchase_Date"] = df1['Purchase_Date'].apply(parser.parse)
# df1['Purchase_Date']
# df1['Purchase_Date_Baru'] = df1['Purchase_Date'].apply(lambda x: try_parsing_date(x, formats))
# df1['Purchase_Date_Baru']
# df1['Purchase_Date_Baru'] = df1['Purchase_Date'].apply(parser.parse)
# df1['Purchase_Date_Baru']

['Purchase_Date']
['BirthDate']
['AdmissionDate', 'DischargeDate', 'AppointmentDate']


In [202]:
# bulan_dict = {
#     'januari': '01', 'februari': '02', 'maret': '03', 'april': '04',
#     'mei': '05', 'juni': '06', 'juli': '07', 'agustus': '08',
#     'september': '09', 'oktober': '10', 'november': '11', 'desember': '12'
# }
# def clean_date(date_str):
#     date_str = str(date_str).lower()
#     for bulan, angka in bulan_dict.items():
#         date_str = date_str.replace(bulan, angka)  # Ganti nama bulan dengan angka
#     return date_str

# df11['DateClean'] = df11['Purchase_Date'].apply(clean_date)
# df11

### Format Country

In [203]:
def apakah_kolom_country(kolom):
    pattern = r'(origin|citizenship|country|nationality)\b|_(origin|citizenship|country|nationality)[_A-Z]?'
    return re.search(pattern, kolom, re.IGNORECASE)

def identifikasi_kolom_country(df):
    kolom_id = []
    for kolom in df.columns:
        ratio = df[kolom].nunique() / len(df)
        if apakah_kolom_country(kolom):
            kolom_id.append(kolom)
    return kolom_id

def apakah_kolom_telp(kolom):
    pattern = r'\b(phone|telp|telephone)|(phone|telp|telephone)\b|_(phone|telp|telephone)[_A-Z]?'
    return re.search(pattern, kolom, re.IGNORECASE)

def identifikasi_kolom_telp(df, threshold = 0.7):
    kolom_id = []
    for kolom in df.columns:
        ratio = df[kolom].nunique() / len(df)
        if (ratio >= threshold) and apakah_kolom_telp(kolom):
            kolom_id.append(kolom)
    return kolom_id

print("kolom country")
print(f"DF 1 {identifikasi_kolom_country(df1)}")
print(f"DF 2 {identifikasi_kolom_country(df2)}")
print(f"DF 3 {identifikasi_kolom_country(df3)}")

kolom country
DF 1 ['Country']
DF 2 ['Nationality']
DF 3 []


In [204]:
print(df1['Country'].unique())
print(df2['Nationality'].unique())

['USA' 'Canada' 'United Kingdom' 'UK' 'CAN' 'usa' 'Usa']
['USA' 'FRANCE' 'Bharat' 'FR' 'America' 'France' 'India' 'IND' 'US'
 'United States']


In [205]:
df1['Country'].unique()

array(['USA', 'Canada', 'United Kingdom', 'UK', 'CAN', 'usa', 'Usa'],
      dtype=object)

In [206]:
country = "usa"
country = country.upper()
print(country)

USA


In [207]:
special_mapping = {
    'america': 'USA',
    'bharat': 'IND',
    'uk' : 'United Kingdom'
}

@lru_cache(maxsize=None)
def format_country(country):
    country = country.lower()
    if country in special_mapping:
        country = special_mapping[country]
    hasil_cari = pycountry.countries.search_fuzzy(country)[0]
    return hasil_cari.name

# def format_country(country):
#     country = country.upper()
#     hasil_cari = pycountry.countries.search_fuzzy(country)
#     if hasil_cari:
#         # Pastikan hasil yang dipilih adalah yang paling tepat, misalnya berdasarkan panjang nama
#         return max(hasil_cari, key=lambda x: len(x.name)).name
#     return country

df1['Country'] = df1['Country'].apply(format_country)
df2['Nationality'] = df2['Nationality'].apply(format_country)
print(df1['Country'].unique())
print(df2['Nationality'].unique())


['United States' 'Canada' 'United Kingdom']
['United States' 'France' 'India']


### Format Nomor Telepon

In [208]:
print("kolom telp")
print(f"DF 1 {identifikasi_kolom_telp(df1)}")
print(f"DF 2 {identifikasi_kolom_telp(df2)}")
print(f"DF 3 {identifikasi_kolom_telp(df3)}")

kolom telp
DF 1 ['Phone_Number']
DF 2 []
DF 3 []


In [209]:
angka = [str(i) for i in range(0,10)]
def ekstrak_simbol(baris):
    simbol = set()
    for nomor in baris:
        for x in nomor:
            if x not in angka and x not in simbol:
                simbol.add(x)
    return re.escape(''.join(simbol))

def format_nomor(nomor, simbol, pemisah="-"):
    clean_text = re.sub(f'[{simbol}]', '', nomor)
    chunk = []
    for i in range(0, len(clean_text), 3):
        chunk.append(clean_text[i:i+3])
    hasil = pemisah.join(chunk)
    return hasil

def panjang_nomor(nomor):
    panjang = set()
    for x in nomor:
        panjang.add(len(x))
    return list(panjang)

In [210]:
hasil = ekstrak_simbol(df1['Phone_Number'])
print(hasil)
print(f'[{hasil}]')
# hasil_panjang = panjang_nomor(df1['Phone_Number'])
# print(hasil_panjang)
df1['Phone_Number'] = df1['Phone_Number'].apply(lambda x:
format_nomor(x, hasil))
df1['Phone_Number']

x\)\-\+\.\ \(
[x\)\-\+\.\ \(]


0              250-518-127-1
1         891-222-899-208-33
2         926-729-991-740-80
3          421-698-421-547-9
4              294-283-628-6
                ...         
10295    785-235-423-889-420
10296     380-450-141-606-93
10297          434-895-981-2
10298          868-251-989-9
10299    258-677-485-319-920
Name: Phone_Number, Length: 10300, dtype: object

### Format Product

In [211]:
def apakah_kolom_product(kolom):
    pattern = r'\b(product|category|produk|kategori)|(product|category|produk|kategori)\b|_(product|category|produk|kategori)[_A-Z]?'
    return re.search(pattern, kolom, re.IGNORECASE)

def identifikasi_kolom_product(df):
    kolom_id = []
    for kolom in df.columns:
        ratio = df[kolom].nunique() / len(df)
        if apakah_kolom_product(kolom):
            kolom_id.append(kolom)
    return kolom_id

print(f"DF 1 {identifikasi_kolom_product(df1)}")
print(f"DF 2 {identifikasi_kolom_product(df2)}")
print(f"DF 3 {identifikasi_kolom_product(df3)}")

DF 1 ['Product_Category', 'Product_Description']
DF 2 []
DF 3 []


In [212]:
df1["Product_Category"].isna().sum()

np.int64(1309)

In [213]:
df1["Product_Category"].unique()

array(['Electronics', 'Clothing', 'Furniture', 'Sports', 'Grocery',
       'Electornics', nan, 'Books'], dtype=object)

In [214]:
# typo_mapping = {
#     'electornics': 'electronics',
# }

# def frequncy_dict(df, kolom):
#     return df[kolom].value_counts().to_dict()

# def format_product(produk):
#     produk = str(produk).lower()
#     if produk in ['na','nan']:
#         produk = 'Unknown'
#     if produk in typo_mapping:
#         produk = typo_mapping[produk]
#     return produk.title()

# df1['Product_Category'] = df1['Product_Category'].apply(format_product)
# df1['Product_Category'].unique()

### Format Age

In [215]:
def apakah_kolom_age(kolom):
    pattern = r'\b(umur|age|usia)|(umur|age|usia)\b|_(umur|age|usia)[_A-Z]?'
    return re.search(pattern, kolom, re.IGNORECASE)

def identifikasi_kolom_age(df):
    kolom_id = []
    for kolom in df.columns:
        ratio = df[kolom].nunique() / len(df)
        if apakah_kolom_age(kolom):
            kolom_id.append(kolom)
    return kolom_id

print(f"DF 1 {identifikasi_kolom_age(df1)}")
print(f"DF 2 {identifikasi_kolom_age(df2)}")
print(f"DF 3 {identifikasi_kolom_age(df3)}")

DF 1 []
DF 2 []
DF 3 ['Age']


In [216]:
df3['Age'].unique()

array([ 81, -88,  94,  26,  35,  25,  54,   0,  80,   1,  10,  21,  36,
        44,   6, -71,  61,  85,  97,  63,  31,  90,  78,  34,  93,  59,
        32,  47,  52, -79,  77,  66,  92,  15,  33,  16,  43,   9,  20,
        83,  24,  56,  45,   3,  79,  65,  51,  57,  95,  72,  84,  71,
        42,  13,  22,  87,  60,   7,   4,  50, -11,  41,  74,  30,  46,
         8,  12,  49,  17,  82,  29,  96,  38,  11,  99,  23,  88,  70,
        73,  40,  19,  28,  53,  98,  86,  76, -95,   5,  91,  69,   2,
        37,  64,  27, -39, -25,  14,  55,  48,  62, -28, -62,  89,  68,
        67,  18, -99,  75, -57,  39,  58, -48, -13, -69, -58, -68, -55,
       -24, -98, -52, -46, -29,  -1, -35, -61, -67, -56, -72, -32, -86,
       -47, -36, -40, -22, -12, -45, -33, -91, -23, -31, -74, -97, -10,
       -37, -83,  -2, -85, -51,  -4, -38, -17,  -8, -92, -96, -77,  -9,
       -21, -20, -63,  -3, -59, -87, -19, -44, -75, -16, -82, -14, -90,
       -26, -50, -27, -78, -65, -15, -53, -49,  -6, -54, -84, -9

In [217]:
def format_age(age):
    age = int(age)
    if age < 0:
        return abs(age)
    if age == 0:
        return age

# kolom_age = identifikasi_kolom_age(df3)
# for kolom in kolom_age:
#     df3[kolom] = df3[kolom].apply(format_age)
# df3['Age'].unique()

### Format Kategorik

In [218]:
country_mapping = {
    'america': 'USA',
    'bharat': 'IND',
    'uk': 'United Kingdom'
}

voting_mapping = {
    'n': 'No', 
    'y': 'Yes',
    ' ': 'Unknown',
    '': 'Unknown',
    np.nan: 'Unknown'
}

marital_mapping = {
    's': 'Single',
    'belummenikah': 'Single', 
    'janda': 'Widowed',
    'duda': 'Widowed',
    'cerai': 'Divorced'
}

education_mapping = {
    'sma': 'High School',
    'hs' : 'High School',
    'phd' : 'Doctorate',
    's1': 'Bachelors',
    's2': 'Master',
    's3': 'Doctorate'
}

gender_mapping = {
    'm': 'Male',
    'f': 'Female',
    'u': 'Unknown',
}
blood_mapping = {
    'positive': '+',
    'negative': '-',
    'positif': '+',
    'negatif': '-',
}
def merge_dictionaries(*dicts):
    merged_dict = {}
    for dictionary in dicts:
        merged_dict.update(dictionary)
    return merged_dict

def apakah_bloodtype(kolom):
    akhiran = kolom.str.endswith('+') | kolom.str.endswith('-')
    ada_kata = kolom.str.contains('positive|negative|positif|negatif', case=False, na=False)
    return akhiran.any() | ada_kata.any()

def format_bloodtype(blood, dictionary):
    blood = blood.lower()
    bagian = blood.split()
    if len(bagian) == 2 and (bagian[0] in ['a', 'b', 'ab', 'o'] and bagian[1] in dictionary):
        darah, tipe = bagian
        tipe = dictionary.get(tipe.lower(), tipe)
        return f"{darah.upper()}{tipe}"
    elif len(bagian) == 1 and bagian[0][-1] in ['+', '-']:
        return bagian[0].upper()
    else:
        return blood.upper()

def replace_values_with_dict(df, dictionary):
    for kolom in df.select_dtypes(include=['object']).columns:
        df[kolom] = df[kolom].str.strip().str.lower()
        df[kolom] = df[kolom].replace(dictionary)
        
        if apakah_bloodtype(df[kolom]):
            df[kolom] = df[kolom].apply(lambda x: format_bloodtype(x, blood_mapping))
        else:
            df[kolom] = df[kolom].str.title()
    return df

merged_dict = merge_dictionaries(
    country_mapping, 
    voting_mapping, 
    marital_mapping,
    education_mapping,
    gender_mapping,
    blood_mapping
    )

df1 = replace_values_with_dict(df1, merged_dict)
df2 = replace_values_with_dict(df2, merged_dict)
df3 = replace_values_with_dict(df3, merged_dict)

In [219]:
df3['BloodType'].unique()

array(['B+', 'O-', 'A+', 'B-', 'A-', 'AB-', 'AB+', 'O+'], dtype=object)

In [220]:
# x = 'A Positive'
# z= x.split()
# a, b= z
# if len(z) == 2 and (bagian[0] in ['a', 'b', 'ab', 'o'] and bagian[1] in dictionary):
#         darah, tipe = bagian
#         tipe = dictionary.get(tipe.lower(), tipe)


In [221]:
df3['BloodType'] = df3['BloodType'].apply(lambda x: format_bloodtype(x, blood_mapping))
df3['BloodType'].unique()

array(['B+', 'O-', 'A+', 'B-', 'A-', 'AB-', 'AB+', 'O+'], dtype=object)

In [222]:
for x in gender_mapping.values():
    print(x)

Male
Female
Unknown


In [223]:
print(df3["Treatment"].unique())
print(df3["Gender"].unique())
print(df3["BloodType"].unique())


['SPECIAL WITH JOB,' 'CERTAINLY WAY TEN,' 'STUDY SMALL,' ...
 'OLD ENVIRONMENT OFFICIAL LEARN,' 'RECENT COMMUNITY ATTORNEY,'
 'SOMEBODY BOTH,']
['Female' 'Male' 'Unknown']
['B+' 'O-' 'A+' 'B-' 'A-' 'AB-' 'AB+' 'O+']


### Format Typos

In [224]:
print(df1['Product_Category'].unique())
print(df3['Diagnosis'].unique())

['Electronics' 'Clothing' 'Furniture' 'Sports' 'Grocery' 'Electornics'
 'Unknown' 'Books']
['Appendicitis' 'Fracture' 'Copd' 'Diabetes' 'Hypertension' 'Hypertensyon'
 'Dyabetes' 'Appendycytys']


In [225]:
# Format Typo
sym_spell = SymSpell(max_dictionary_edit_distance=4, prefix_length=7)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt")
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
# @lru_cache(maxsize=None)
def format_typo(typo):
    hasil = str(typo)
    saran_typo = sym_spell.lookup(hasil, Verbosity.CLOSEST, max_edit_distance=4)
    if saran_typo:
        return saran_typo[0].term.title()
    return hasil.title()

In [226]:
df1['Product_Category'] = df1['Product_Category'].apply(format_typo)
df3['Diagnosis'] = df3['Diagnosis'].apply(format_typo)
print(df1['Product_Category'].unique())
print(df3['Diagnosis'].unique())

['Electronics' 'Clothing' 'Furniture' 'Sports' 'Grocery' 'Unknown' 'Books']
['Appendicitis' 'Fracture' 'Top' 'Diabetes' 'Hypertension']


In [227]:
sym_spell = SymSpell(max_dictionary_edit_distance=4, prefix_length=10)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt")
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

kata_benar = {
}
def format_typo(typo):
    if typo in kata_benar:
        return kata_benar[typo].title()
    saran_typo = sym_spell.lookup(typo, Verbosity.CLOSEST, max_edit_distance=4)
    if saran_typo:
        jawaban_benar = saran_typo[0].term.title()
    else: 
        jawaban_benar = typo.title()
    kata_benar[typo] = jawaban_benar
    return jawaban_benar

def correct_typo(series):
    return series.apply(format_typo)

# sym_spell = SymSpell(max_dictionary_edit_distance=4, prefix_length=7)
# dictionary_path = pkg_resources.resource_filename(
#     "symspellpy", "frequency_dictionary_en_82_765.txt")
# sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
# @lru_cache(maxsize=None)
# def format_typo(typo):
#     hasil = str(typo)
#     saran_typo = sym_spell.lookup(hasil, Verbosity.CLOSEST, max_edit_distance=4)
#     if saran_typo:
#         return saran_typo[0].term.title()
#     return hasil.title()

# df1['Product_Category'] = df1['Product_Category'].apply(format_typo)
df1['Product_Category'] = correct_typo(df1['Product_Category'])
df3['Diagnosis'] = correct_typo(df3['Diagnosis'])
print(df1['Product_Category'].unique())
# df3['Diagnosis'] = df3['Diagnosis'].apply(format_typo)
print(df3['Diagnosis'].unique())

['Electronics' 'Clothing' 'Furniture' 'Sports' 'Grocery' 'Unknown' 'Books']
['Appendicitis' 'Fracture' 'Top' 'Diabetes' 'Hypertension']


In [228]:
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=10)
typo = 'copd'
saran_typo = sym_spell.lookup(typo, Verbosity.CLOSEST, max_edit_distance=2)
print(saran_typo)

[]


In [229]:
# spell = SpellChecker()
# @lru_cache(maxsize=None)
# def format_typo(typo):
#     hasil = str(spell.correction(typo))
#     if hasil.lower() == 'none':
#         return f"{typo} ini woy"
#     return hasil.title()

# df1['Product_Category'] = df1['Product_Category'].apply(format_typo)
# print(df1['Product_Category'].unique())

In [230]:
def apakah_bloodtype(kolom):
    akhiran = kolom.str.endswith('+') | kolom.str.endswith('-')
    # ada_kata = kolom.str.contains('positive|negative|positif|negatif', case=False, na=False)
    return akhiran.any()

hasil = apakah_bloodtype(df3['Medication'])
print(hasil)

True


In [231]:
def process_missing_dict_values(series, dictionary, process_func):
    """
    Check if values in a series are not in dictionary and apply a function
    
    Parameters:
    series: pandas Series to check
    dictionary: dictionary containing valid values
    process_func: function to apply to values not in dictionary
    """
    # Create mask for values not in dictionary
    mask = ~series.str.lower().isin([k.lower() for k in dictionary.keys()])
    
    # Apply function only to values not in dictionary
    series.loc[mask] = series.loc[mask].apply(process_func)
    
    return series

def replace_values_with_dict(df, dictionary):
    kolom_darah = [col for col in df.columns if apakah_bloodtype(df[col])]
    df = df.apply(lambda x: x.str.strip().str.lower() if x.dtype == 'object' else x)
    
    for kolom in df.select_dtypes(include=['object']).columns:
        # First replace values using dictionary
        df[kolom] = df[kolom].replace(dictionary)
        
        # Apply format_typo only to values not in dictionary
        df[kolom] = process_missing_dict_values(df[kolom], dictionary, format_typo)
        
        if kolom in kolom_darah:
            df[kolom] = df[kolom].apply(lambda x: format_bloodtype(x, blood_mapping))
        else:
            df[kolom] = df[kolom].str.title()
            
    return df

In [232]:
country_mapping = {
    'america': 'USA',
    'bharat': 'IND',
    'uk': 'United Kingdom'
}
voting_mapping = {
    'n': 'No', 
    'y': 'Yes',
    ' ': 'Unknown',
    '': 'Unknown',
    np.nan: 'Unknown'
}
marital_mapping = {
    's': 'Single',
    'belummenikah': 'Single', 
    'janda': 'Widowed',
    'duda': 'Widowed',
    'cerai': 'Divorced'
}
education_mapping = {
    'sma': 'High School',
    'hs' : 'High School',
    'phd' : 'Doctorate',
    's1': 'Bachelors',
    's2': 'Master',
    's3': 'Doctorate'
}
gender_mapping = {
    'm': 'Male',
    'f': 'Female',
    'u': 'Unknown',
}
blood_mapping = {
    'positive': '+',
    'negative': '-',
    'positif': '+',
    'negatif': '-',
}

def merge_dictionaries(*dicts):
    merged_dict = {}
    for dictionary in dicts:
        merged_dict.update(dictionary)
    return merged_dict

def apakah_bloodtype(kolom):
    akhiran = kolom.str.endswith('+') | kolom.str.endswith('-')
    ada_kata = kolom.str.contains('positive|negative|positif|negatif', case=False, na=False)
    return akhiran.any() | ada_kata.any()

def format_bloodtype(blood, dictionary):
    output = blood
    bloods = blood.lower()
    bagian = bloods.split()
    if len(bagian) == 2 and (bagian[0] in ['a', 'b', 'ab', 'o'] and bagian[1] in dictionary):
        darah, tipe = bagian
        tipe = dictionary.get(tipe.lower(), tipe)
        return f"{darah.upper()}{tipe}"
    elif len(bagian) == 1 and bagian[0][-1] in ['+', '-']:
        return bagian[0].upper()
    else:
        return output

# Format Typo
spell = SpellChecker()
@lru_cache(maxsize=None)
def format_typo(typo):
    hasil = str(spell.correction(typo))
    if hasil.lower() == 'none':
        return typo
    return hasil.title()

def process_missing_dict_values(series, dictionary, process_func):
    """
    Check if values in a series are not in dictionary and apply a function
    
    Parameters:
    series: pandas Series to check
    dictionary: dictionary containing valid values
    process_func: function to apply to values not in dictionary
    """
    # Create mask for values not in dictionary
    mask = ~series.str.lower().isin([k.lower() for k in dictionary.keys()])
    
    # Apply function only to values not in dictionary
    series.loc[mask] = series.loc[mask].apply(process_func)
    
    return series

def replace_values_with_dict(df, dictionary):
    kolom_darah = [col for col in df.columns if apakah_bloodtype(df[col])]
    df = df.apply(lambda x: x.str.strip().str.lower() if x.dtype == 'object' else x)
    
    for kolom in df.select_dtypes(include=['object']).columns:
        # First replace values using dictionary
        df[kolom] = df[kolom].replace(dictionary)
        
        # Apply format_typo only to values not in dictionary
        df[kolom] = process_missing_dict_values(df[kolom], dictionary, format_typo)
        
        if kolom in kolom_darah:
            df[kolom] = df[kolom].apply(lambda x: format_bloodtype(x, blood_mapping))
        else:
            df[kolom] = df[kolom].str.title()
            
    return df

merged_dict = merge_dictionaries(
    country_mapping, 
    voting_mapping, 
    marital_mapping,
    education_mapping,
    gender_mapping,
    blood_mapping
    )


### Buang Kolom Tidak Perlu

In [ ]:
def format_kolom_hapus(df, threshold = 0.6):
    kolom_id = []
    for kolom in df.select_dtypes(include=['object']).columns:
        ratio = df[kolom].nunique() / len(df)
        jumlah = df[kolom].nunique()
        # print(ratio)
        if ratio >= threshold or jumlah > 100:
            kolom_id.append(kolom)
    return kolom_id

print(df3['Medication'].nunique())
print(f"DF 1 {format_kolom_hapus(df1)}")
print(f"DF 2 {format_kolom_hapus(df2)}")
print(f"DF 3 {format_kolom_hapus(df3)}")

2665
DF 1 ['Purchase_Date', 'Product_Description', 'Phone_Number', 'Email']
DF 2 ['Name', 'Address', 'PassportNumber']
DF 3 ['AdmissionDate', 'DischargeDate', 'Treatment', 'Doctor', 'Medication', 'TestResults', 'AppointmentDate']


In [59]:
df3['AdmissionDate']

0         2024-10-03
1         2023-06-27
2         2024-05-18
3        Mar 11 1978
4         2024-11-23
            ...     
10295     2024-03-16
10296     2023-03-29
10297    Oct 11 1998
10298     2024-02-01
10299    Mar 31 2017
Name: AdmissionDate, Length: 10300, dtype: object

In [58]:
for kolom in df1.select_dtypes(include=['object']).columns:
    print(kolom)
for kolom in df2.select_dtypes(include=['object']).columns:
    print(kolom)
for kolom in df3.select_dtypes(include=['object']).columns:
    print(kolom)
print(df3['AdmissionDate'].dtype)
print(df3['DischargeDate'].dtype)
print(df3['AppointmentDate'].dtype)


Product_Category
Purchase_Date
Product_Description
Country
Phone_Number
Email
Name
Nationality
Address
VotingStatus
PassportNumber
MaritalStatus
CriminalRecord
EducationLevel
AdmissionDate
DischargeDate
Diagnosis
Treatment
Doctor
Department
Gender
BloodType
Medication
TestResults
InsuranceProvider
AppointmentDate
object
object
object


## Setelah Format

In [245]:
df1_baru = pd.read_csv("clean-custdirty_data.csv")
df2_baru = pd.read_csv("clean-government_citizenship_dirty.csv")
df3_baru = pd.read_csv("clean-hospital_data_dirty.csv")

# df1_baru

In [246]:
# df2_baru

In [247]:
df3_baru

,AdmissionDate,DischargeDate,Diagnosis,Department,Age,Gender,BloodType,BillingAmount,InsuranceProvider,AppointmentDate
0,2024-10-03,2025-05-11,Appendicitis,Orthopedics,81,Female,Be,9495.73,Unknown,2024-02-17
1,2023-06-27,2023-12-05,Fracture,End,88,M,Of,8681.08,Coinsurance,2023-08-08
2,2024-05-18,2025-04-29,Appendicitis,Orthopedics,94,M,Of,2829.42,Unknown,2024-05-21
3,1978-03-11,2019-10-26,Cold,Endocrinology,26,F,A,1121.27,Unknown,2024-11-03
4,2024-11-23,2024-12-02,Diabetes,Orth,35,Female,Be,5555.75,Coinsurance,2023-11-14
...,...,...,...,...,...,...,...,...,...,...
7025,2024-11-06,2025-03-01,Fracture,Endocrinology,60,F,A,5537.98,Coinsurance,2024-12-03
7026,2023-07-19,2024-09-16,Cold,Endocrinology,3,Male,Appositive,3703.67,Healthplus,2024-07-03
7027,2023-04-18,2023-06-08,Diabetes,Cardie,90,U,Of,5659.97,Unknown,2024-07-29
7028,2024-01-03,2024-11-14,Cold,Endocrinology,35,F,Negative,3511.16,Unknown,2024-12-14


In [248]:
df3_baru['Diagnosis'] = df3_baru['Diagnosis'].apply(format_typo)
df3_baru['Diagnosis'].unique()

array(['Appendicitis', 'Fracture', 'Cold', 'Diabetes', 'Hypertension',
       'Appendycytys'], dtype=object)

In [249]:
for kolom in df1_baru.select_dtypes(include=['object']).columns:
    print(kolom)
for kolom in df2_baru.select_dtypes(include=['object']).columns:
    print(kolom)
for kolom in df3_baru.select_dtypes(include=['object']).columns:
    print(kolom)
# print(df3_baru['AdmissionDate'].dtype)
# print(df3_baru['DischargeDate'].dtype)
print(df3_baru['AppointmentDate'].dtype)

# df3_baru['AppointmentDate'].isna().sum() 
# df3_baru['AppointmentDate'] 

def safe_parse(date_str):
    date_str = str(date_str).strip()
    try:
        parsed_date = parser.parse(date_str)
        return pd.to_datetime(parsed_date)
    except (ValueError, TypeError): 
        return pd.NaT 

df3_baru['AppointmentDate'] = df3_baru['AppointmentDate'].apply(safe_parse)
df3_baru['AppointmentDate']

Product_Category
Purchase_Date
Product_Description
Country
Phone_Number
Email
Name
BirthDate
Nationality
Address
VotingStatus
PassportNumber
MaritalStatus
CriminalRecord
EducationLevel
AdmissionDate
DischargeDate
Diagnosis
Department
Gender
BloodType
InsuranceProvider
AppointmentDate
object


0      2024-02-17
1      2023-08-08
2      2024-05-21
3      2024-11-03
4      2023-11-14
          ...    
7025   2024-12-03
7026   2024-07-03
7027   2024-07-29
7028   2024-12-14
7029   2024-10-11
Name: AppointmentDate, Length: 7030, dtype: datetime64[ns]

In [250]:
def tes(df):
    for kolom in df.columns:
        ratio = df[kolom].nunique() / len(df)
        print(ratio)
        print(kolom)
tes(df3_baru)

0.20327169274537696
AdmissionDate
0.19203413940256045
DischargeDate
0.0008534850640113798
Diagnosis
0.0008534850640113798
Department
0.01422475106685633
Age
0.0007112375533428165
Gender
0.0008534850640113798
BloodType
0.9706970128022759
BillingAmount
0.0007112375533428165
InsuranceProvider
0.10398293029871977
AppointmentDate
